### Global params

In [1]:
%load_ext autoreload
import pandas as pd
k_closest_users = 2   # Number of closest-users

### Pre-process

In [2]:
%%time
%autoreload 2
from ratingmatrix import rating_matrix
from user_keywords import load_content_data, load_events_data, merge_on_docID, get_user_counts, get_user_profile

df_content = load_content_data('data/content_refine')
df_content = df_content.rename(columns={"id": "documentId"})
# TODO: using num_days for testing
df_events = load_events_data('data/active1000', num_days=None)
df_events = df_events.drop(['title', 'publishtime'], axis=1)
df_merged = merge_on_docID(df_events, df_content)
df_user_frequencies = get_user_counts(df_merged)
df_user_profile = get_user_profile(df_user_frequencies)
df_user_profile.to_pickle('data/matrix-df/user_profiles.pkl')
df_user_item = rating_matrix(df_events)
df_user_item.to_pickle('data/matrix-df/user_item.pkl')


Wall time: 1min 11s


### Load preprocessed matrices + train_test

In [3]:
%autoreload 2
from ratingmatrix import rating_matrix_train_test_split

df_user_profile = pd.read_pickle('data/matrix-df/user_profiles.pkl')
df_user_profile.head()

df_user_item = pd.read_pickle('data/matrix-df/user_item.pkl')
df_user_item.head()

index = df_user_item.index
columns = df_user_item.columns
train, test = rating_matrix_train_test_split(df_user_item.to_numpy(), fraction=0.2)

# Train and test are disjoint (in terms of the 1-values)
train = pd.DataFrame(data=train, index=index, columns=columns)
test = pd.DataFrame(data=test, index=index, columns=columns)

## Method


In [4]:
%%time
from recommender_methods import collaborative_filtering_user_based

# List with users and their recommended articles
recommendations = []

# The number of requests for recommendations
num_recommendations = 0

# Returns a list with recommended articles
#for user in range(len(df_user_profile.index)):
for user in range(10):
    num_recommendations += 1
    recommended = collaborative_filtering_user_based(df_user_profile, train, train.index[user], k_closest_users)
    recommendations.append(recommended)

predict = test.copy(deep=True)
predict[:] = 0.0
for user in recommendations:
    for article in user['articles']:
        predict.at[user['user_id'], article] = 1.0

predict.head()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\Krist\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\magics\execution.py", line 1321, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 13, in <module>
  File "c:\Users\Krist\Dokumenter\Skole\github\recommender-system\recommender_methods.py", line 9, in collaborative_filtering_user_based
    with_pearson = rm.corrwith(user_id_row, axis=1)
  File "C:\Users\Krist\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py", line 8571, in corrwith
    return this.apply(lambda x: other.corr(x, method=method), axis=axis)
  File "C:\Users\Krist\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py", line 7765, in apply
    return op.get_result()
  File "C:\Users\Krist\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\apply.py", line 185,

TypeError: object of type 'NoneType' has no len()

In [7]:
from evaluation import evaluate
evaluate(recommendations, test, 5)
evaluate(recommendations, test, 10)
evaluate(recommendations, test, 20)
evaluate(recommendations, test, 100)
evaluate(recommendations, test)






Average recall@5 is 0.0029
ARHR@5 is 0.2097
CTR@5 is 7.9200%
Real CTR@5 is 39.6000%

Average recall@10 is 0.0051
ARHR@10 is 0.2450
CTR@10 is 6.7100%
Real CTR@10 is 67.1000%

Average recall@20 is 0.0096
ARHR@20 is 0.2845
CTR@20 is 6.3450%
Real CTR@20 is 126.9000%

Average recall@100 is 0.0480
ARHR@100 is 0.3870
CTR@100 is 6.3350%
Real CTR@100 is 633.5000%

Average recall is 0.4284
ARHR is 0.5240
CTR is 6.2512%
Real CTR is 5883.9000%
